In [6]:
!pip install opencv-python tensorflow keras mtcnn numpy


You should consider upgrading via the 'c:\users\arunnachalam\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [7]:
import numpy as np
import cv2
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [8]:
def load_dataset(data_path):
    images = []
    labels = []
    label_map = {}
    label_counter = 0

    for label_name in os.listdir(data_path):
        label_dir = os.path.join(data_path, label_name)
        if os.path.isdir(label_dir):
            label_map[label_counter] = label_name
            for image_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, image_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (224, 224))  # Resize for ResNet50
                images.append(img)
                labels.append(label_counter)
            label_counter += 1

    images = np.array(images, dtype='float32') / 255.0
    labels = to_categorical(labels)
    return images, labels, label_map

In [12]:
def load_dataset(data_path):
    images = []
    labels = []
    label_map = {}
    label_counter = 0

    for label_name in os.listdir(data_path):
        label_dir = os.path.join(data_path, label_name)
        if os.path.isdir(label_dir):
            label_map[label_counter] = label_name
            for image_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, image_name)
                img = cv2.imread(img_path)
                
                # Check if the image is read properly
                if img is None:
                    print(f"Warning: Skipping unreadable image {img_path}")
                    continue
                
                img = cv2.resize(img, (224, 224))  # Resize for ResNet50
                images.append(img)
                labels.append(label_counter)
            label_counter += 1

    images = np.array(images, dtype='float32') / 255.0
    labels = to_categorical(labels)
    return images, labels, label_map


In [17]:
def load_dataset(data_path):
    images = []
    labels = []
    label_map = {}
    label_counter = 0

    for label_name in os.listdir(data_path):
        label_dir = os.path.join(data_path, label_name)
        if os.path.isdir(label_dir):
            label_map[label_counter] = label_name
            print(f"Processing label: {label_name} ({label_counter})")  # Debugging line
            for image_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, image_name)
                img = cv2.imread(img_path)
                
                # Check if the image is read properly
                if img is None:
                    print(f"Warning: Skipping unreadable image {img_path}")
                    continue
                
                # Resize the image for ResNet50
                img = cv2.resize(img, (224, 224))
                images.append(img)
                labels.append(label_counter)

            label_counter += 1

    # Check if images list is empty
    if len(images) == 0:
        raise ValueError("No images found in the dataset. Please check the dataset directory.")

    images = np.array(images, dtype='float32') / 255.0
    labels = to_categorical(labels)
    
    return images, labels, label_map


In [13]:
def build_resnet50_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
data_path = 'C:/Users/Arunnachalam/Downloads/New folder (3)/faces'
X, y, label_map = load_dataset(data_path)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = build_resnet50_model(input_shape=(224, 224, 3), num_classes=len(label_map))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
datagen.fit(X_train)

history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=10, validation_split=0.1)

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
datagen_train = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
datagen_train.fit(X_train)

datagen_val = ImageDataGenerator()  # No augmentation for validation
datagen_val.fit(X_test)

In [23]:
model = build_resnet50_model(input_shape=(224, 224, 3), num_classes=len(label_map))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    datagen_train.flow(X_train, y_train, batch_size=8),
    epochs=10,
    validation_data=datagen_val.flow(X_test, y_test, batch_size=8)
    )

Epoch 1/10


3/3 [==============================] - 28s 6s/step - loss: 0.7269 - accuracy: 0.4000 - val_loss: 0.5766 - val_accuracy: 0.8333
Epoch 2/10
3/3 [==============================] - 7s 2s/step - loss: 0.6930 - accuracy: 0.6000 - val_loss: 0.4750 - val_accuracy: 0.8333
Epoch 3/10
3/3 [==============================] - 7s 3s/step - loss: 0.7476 - accuracy: 0.6000 - val_loss: 0.4694 - val_accuracy: 0.8333
Epoch 4/10
3/3 [==============================] - 7s 2s/step - loss: 0.6904 - accuracy: 0.6000 - val_loss: 0.5557 - val_accuracy: 0.8333
Epoch 5/10
3/3 [==============================] - 7s 3s/step - loss: 0.6799 - accuracy: 0.6000 - val_loss: 0.6952 - val_accuracy: 0.3333
Epoch 6/10
3/3 [==============================] - 7s 3s/step - loss: 0.7065 - accuracy: 0.5000 - val_loss: 0.6914 - val_accuracy: 0.3333
Epoch 7/10
3/3 [==============================] - 7s 2s/step - loss: 0.7041 - accuracy: 0.4500 - val_loss: 0.6357 - val_accuracy: 0.8333
Epoch 8/10
3/3 [======================

In [25]:
# Step 5: Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Validation Accuracy: {test_acc:.2f}")

1/1 [==============================] - 2s 2s/step - loss: 0.5208 - accuracy: 0.8333
Validation Accuracy: 0.83


In [26]:
model.save('face_recognition_resnet50.h5')

c:\Users\Arunnachalam\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
def recognize_face(model, image_path, label_map):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0) / 255.0
    prediction = model.predict(img)
    label = np.argmax(prediction)
    print(f"Predicted label: {label_map[label]}")
recognize_face(load_model('face_recognition_resnet50.h5'), 'C:/Users/Arunnachalam/Downloads/New folder (3)/faces/Bill_Gates/Bill_Gates_0001.jpg', label_map)


1/1 [==============================] - 5s 5s/step
Predicted label: Bill_Gates


In [30]:
import numpy as np
import cv2
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Step 1: Load and preprocess the dataset
def load_dataset(data_path):
    images = []
    labels = []
    label_map = {}
    label_counter = 0

    for label_name in os.listdir(data_path):
        label_dir = os.path.join(data_path, label_name)
        if os.path.isdir(label_dir):
            label_map[label_counter] = label_name
            print(f"Processing label: {label_name} ({label_counter})")  # Debugging line
            for image_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, image_name)
                img = cv2.imread(img_path)
                
                # Check if the image is read properly
                if img is None:
                    print(f"Warning: Skipping unreadable image {img_path}")
                    continue
                
                # Resize the image for ResNet50
                img = cv2.resize(img, (224, 224))
                images.append(img)
                labels.append(label_counter)

            label_counter += 1

    # Check if images list is empty
    if len(images) == 0:
        raise ValueError("No images found in the dataset. Please check the dataset directory.")

    images = np.array(images, dtype='float32') / 255.0
    labels = to_categorical(labels)
    
    return images, labels, label_map

# Step 2: Build ResNet50 model
def build_resnet50_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Step 3: Train the model
data_path = 'C:/Users/Arunnachalam/Downloads/New folder (3)/faces'  # Update this to your dataset directory
X, y, label_map = load_dataset(data_path)

# Split the data manually
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the ImageDataGenerators
datagen_train = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)
datagen_train.fit(X_train)

datagen_val = ImageDataGenerator()  # No augmentation for validation
datagen_val.fit(X_val)

# Create and compile the model
model = build_resnet50_model(input_shape=(224, 224, 3), num_classes=len(label_map))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    datagen_train.flow(X_train, y_train, batch_size=32),
    epochs=1,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=32)
)

# Step 4: Evaluate the model
test_loss, test_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {test_acc:.2f}")

# Step 5: Save the model
model.save('face_recognition_resnet501.h5')

# Step 6: Test on a single image (optional)
def recognize_face(model, image_path, label_map):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0) / 255.0
    prediction = model.predict(img)
    label = np.argmax(prediction)
    print(f"Predicted label: {label_map[label]}")

# Example usage
recognize_face(load_model('face_recognition_resnet501.h5'), 'C:/Users/Arunnachalam/Downloads/New folder (3)/faces/Bill_Gates/Bill_Gates_0001.jpg', label_map)


Processing label: Bill_Gates (0)
Processing label: Bill_Graham (1)
1/1 [==============================] - 1s 1s/step - loss: 0.5869 - accuracy: 0.8333
Validation Accuracy: 0.83


c:\Users\Arunnachalam\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 4s 4s/step
Predicted label: Bill_Gates
